In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Note:** This notebook can run using TensorFlow 2.5.0

In [1]:
import requests
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

In [12]:
tokenizer = Tokenizer()

# sonnets.txt
# !gdown --id 108jAePKK4R3BVYBbYJZ32JWUwxeMg20K

# data = open('./sonnets.txt').read()

# importing data from web as text
url = 'https://raw.githubusercontent.com/martin-gorner/tensorflow-rnn-shakespeare/master/shakespeare/sonnets.txt'
data = requests.get(url).text

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [4]:
### START CODE HERE
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))# Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences = True)))# An LSTM Layer)
model.add(Dropout(0.2))# A dropout layer)
model.add(LSTM(100))# Another LSTM Layer)
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))# A Dense Layer including regularizers)
model.add(Dense(total_words, activation='softmax'))# A Dense Layer)
# Pick an optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])# Pick a loss function and an optimizer)
### END CODE HERE

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 100)            400       
_________________________________________________________________
bidirectional (Bidirectional (None, 2, 300)            301200    
_________________________________________________________________
dropout (Dropout)            (None, 2, 300)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
Total params: 462,214
Trainable params: 462,214
Non-trainable params: 0
__________________________________________________

In [5]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 8s 8s/step - loss: 1.4274 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 1.4163 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 16ms/step - loss: 1.4041 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 1.3920 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 1.3796 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 19ms/step - loss: 1.3641 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 15ms/step - loss: 1.3497 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 1.3316 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 21ms/step - loss: 1.3119 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 18ms/step - loss: 1.2932 - accuracy: 0.5000
Epoch 11/100
1/1 [===

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)